In [ ]:
!pip install requests
!pip install pandas 
import requests

import time
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import numpy as np
import json

pd.set_option("display.max_rows", 50)
pd.set_option("display.max_columns", 50)

In [ ]:
 !pip install openpyxl

# Загрузка данных из таблицы НКР

In [ ]:
# Загрузка из таблицы НКР
df = pd.read_excel ('') # вставить адрес расположения файла
df

# Разделяем исходный файл на 2 части

In [ ]:
# фильтруем адреса без квартир, т.к. данных по ним можно найти используя ФИАС дома 
df_no_flat = df[df['Квартира ГОРОД'].isnull()]
df_no_flat.to_excel('Без квартир обработатать.xlsx', index=False)

# фильтруем адреса с квартирами 
df_flat = df[df['Квартира ГОРОД'].notnull()]
df_flat.to_excel('с квартирами обработатать.xlsx', index=False)


# Поиск данных для домов без квартир

In [ ]:
# Загрузка домов без квартир, по которым нужно найти ГАР
df_no_flat
#df_no_flat = pd.read_excel ('') #если работаем с с разделенным файлом



In [ ]:
# Для файла с домами без квартир запускаем поиск по ФИАС. 

for i, row in df_no_flat.iloc [0:10000, :].iterrows():
    d = row ['ФИАС дома ГОРОД']
    
    headers = {
        'accept': 'application/json',
        'master-token': '', # записать токен
              }

    params = {
        'object_guid': d,
        'address_type': '2',
              }

    response = requests.get(
        'https://fias-public-service.nalog.ru/api/spas/v2.0/GetAddressItemByGuid',
        params=params,
        headers=headers,
             )
    
    json = response.json()
    
    #данные по адресу
    full_name_address = json['addresses'][0]['full_name'] # полная строка адреса (муниципальное деление)
    cadastral_number_inf = json['addresses'][0]['address_details']['cadastral_number'] #кадастровый номер
    object_guid_inf = json['addresses'][0]['object_guid'] # код ГАР
        
    # Записываем найденный адрес   
    df_no_flat.loc [i,'FIAS'] = object_guid_inf 
    df_no_flat.loc [i,'caddastral_number'] = cadastral_number_inf
    df_no_flat.loc [i,'full_name_address'] = full_name_address
    
    time.sleep(0.005)
df_no_flat

In [ ]:
# фильтрация для записи в файл

df_no_flat = df_no_flat[df['FIAS'] == 'nan']
df_no_flat.to_excel('НЕ обработан файл 2025_02_24 часть7.xlsx', index=False)

df_no_flat2 = df_no_flat[df_no_flat['FIAS'] != 'nan']
df_no_flat2.to_excel('обработан файл 2024_02_25 часть 7.xlsx', index=False)

# Поиск данных для домов с квартирами

In [ ]:
# Загрузка домов с квартирами, по которым нужно найти ГАР
df_flat
#df_flat = pd.read_excel ()



In [ ]:
# Ищем по ФИАС дома полную адресную строку
for i, row in df_flat.iloc [0:10000, :].iterrows():
    d = row ['ФИАС дома ГОРОД']
    headers = {
        'accept': 'application/json',
        'master-token': '', # указать токен
                }

    params = {
        'object_guid': d,
        'address_type': '1',
              }
    response = requests.get(
        'https://fias-public-service.nalog.ru/api/spas/v2.0/GetAddressItemByGuid',
        params=params,
        headers=headers,
             )
    
    json = response.json()
    
    #Получаем полную адресную строку
    full_name_address_house = json['addresses'][0]['full_name']  # полная строка адреса (муниципальное деление)
        
    #записываем полную адресную строку дома в df
        
    df_flat.loc [i,'full_name_address_house'] = full_name_address_house

In [ ]:
# объединяем колонки для получения полного адреса с квартирой
df_flat['full_name_address_house'] = df_flat['full_name_address_house'] +', квартира ' + df_flat['Квартира ГОРОД']

In [ ]:
# Получаем из налоговой информацию по адресам. Может быть два адреса похожих по написанию. Из ответа налоговой получаем запись о двух адресах.
for i, row in df_flat.iloc [0:10000, :].iterrows():
    d = row ['full_name_address_house']
    headers = {
                'accept': 'application/json',
                 'master-token': '', # указать токен
                }
    params = {
            'search_string': d,
            'address_type': '2', #Вид представления адреса: 1 - административное деление, 2 - муниципальное деление
             }
    response = requests.get('https://fias-public-service.nalog.ru/api/spas/v2.0/SearchAddressItems', 
                            params=params, headers=headers)
    json = response.json()
    
    #первый найденный адрес по строке (чаще всего земельный участок)
    object_guid_inf = json['addresses'][0]['object_guid'] # код ГАР
    full_name_address = json['addresses'][0]['full_name'] # полная строка адреса (муниципальное деление)
    cadastral_number_inf = json['addresses'][0]['address_details']['cadastral_number'] #кадастровый номер
    
    #второй найденный адрес по строке 
    object_guid_inf_1 = json['addresses'][1]['object_guid'] # код ГАР
    full_name_address_1 = json['addresses'][1]['full_name'] # полная строка адреса (муниципальное деление)
    cadastral_number_inf_1 = json['addresses'][1]['address_details']['cadastral_number'] #кадастровый номер
    
    # Записываем первый найденный адрес   
    df_flat.loc [i,'FIAS'] = object_guid_inf 
    df_flat.loc [i,'caddastral_number'] = cadastral_number_inf
    df_flat.loc [i,'full_name_address'] = full_name_address
    
    # Записываем второй найденный адрес
    df_flat.loc [i,'FIAS_1'] = object_guid_inf_1 
    df_flat.loc [i,'caddastral_number_1'] = cadastral_number_inf_1
    df_flat.loc [i,'full_name_address_1'] = full_name_address_1
 
    time.sleep(0.02)
df_flat


In [ ]:
# фильтрация для записи в файл

df_flat = df_flat[df_flat['FIAS'] == 'nan']
df_flat.to_excel('НЕ обработан файл.xlsx', index=False)

df_flat2 = df_flat[df_flat['FIAS'] != 'nan']
df_flat2.to_excel('обработан файл.xlsx', index=False)